In [ ]:
%pip install autoscraper

In [ ]:
import requests
from ipylab import JupyterFrontEnd, Panel
import difflib as dili
from datetime import date
from ipywidgets import interact, fixed
from ipytree import Node, Tree
from autoscraper import AutoScraper
import itertools as it
from requests.exceptions import TooManyRedirects

def db():
    gojson = 'https://getpantry.cloud/apiv1/pantry/3662f943-14e1-4185-8e1b-17657cfb3959'
    # 'https://db.neelr.dev/api/7a4ae2c5ad8234b9be6a169a2028e6e6'
    return gojson

In [2]:
def gettext(url, pagination=''):
    r = requests.get(url)
    if pagination == '':
        return [r.text]
    pages = []
    status = r.status_code
    index = 1
    while status == 200:
        txt = r.text
        pages.append(txt)
        index += 1
        parts = url.split('?')
        page = f'{parts[0]}/{pagination}/{index}/?{parts[1]}'
        r = requests.get(page)
        status = r.status_code
    return pages


In [3]:
TYPE_NONE = 0
TYPE_POST = 1
TYPE_HRMASTER = 2
TYPE_BNREF = 3
TYPE_SMART = 4
TYPE_ARCHIVE = 5

def path():
    return '/basket/' # jobs

def key(url, part=0):
    return url.split('//')[1].split('/')[part]

def append(url, wanted, callback=TYPE_NONE):
    ep = db() + path() + key(url)
    r = requests.get(ep)
    time.sleep(1)
    if r.status_code == 200:
        return
    scraper = AutoScraper()
    try:
        pages = gettext(url, pagination='page' if key(url).startswith('medicover') else '')
        result = []
        for page in pages:
            result.append(scraper.build(html=page, wanted_list=wanted))
        result = [item for sublist in result for item in sublist]
    except TooManyRedirects:
        pass
    try:
        if len(scraper.stack_list) == 0:
            function = [None, 'cb_post', 'cb_hrmaster', 'cb_bnref', 'cb_smart', 'cb_archive'][callback]
            data = {'autoscraper': {}, 'result': globals()[function](key(url, 2 if callback == TYPE_ARCHIVE else 0)), 'callback': function}
        else:
            data = {'autoscraper': dict(scraper.stack_list[0]), 'result': result}
    except KeyError:
        print('Error: Scraper failed to build list')
    else:
        if callback == TYPE_ARCHIVE:
            ep = db() + path() + key(data['result'][-1])
            archive = requests.get(ep)
            time.sleep(1)
            if archive.status_code == 200:
                return
        requests.post(ep, json=data)
        time.sleep(1)

In [4]:
import json
from bs4 import BeautifulSoup
from html import unescape
import unicodedata

def cb_post(key):
    subtype = 0 if key.split('.')[1] == 'posta' else 1
    if subtype == 0:
        endpoint = 'intellec'
        query = 'county[]=Budapest&'
        routeQuery = 'category[]=Szellemi'
    else:
        endpoint = '/public_job_elist'
        query = 'content=Budapest&'
        routeQuery = ''
    url = 'https://' + key + '/jsbq' + endpoint
    p = requests.post(url, data={'ajax': 1, 'init': 1, 'isCart': 0, 'isHighlightedList': 0, 'isSimilarSearch': 0, 'q': query, 'routeQuery': routeQuery})
    data = []
    if p.status_code == 200:
        data = [row['url'] for row in p.json()['rows']]
    return data
def cb_hrmaster(key):
    url = 'https://' + key + '/Datacenter/Registration'
    p = requests.post(url + '/GetPositionsBySearchDto',
                      json={"searchDto":{"searchText":"","careerLevelTypeIdList":[],"locationOfWorkTypeIdList":[],"styleOfWorkTypeIdList":[],"positionGroupIdList":[],
                                         "qualificationTypeIdList":[],"categoryTypeIdList":[],"languageWithLevelIdList":[]#,"isLanguageConnectionAnd":false,"positionGroupId":null
                                        }})
    data = []
    if p.status_code == 200:
        rescraper = AutoScraper()
        data = rescraper.build(html=p.text, text_fuzz_ratio=0.64, wanted_list=[url + '/JobAdvertisement//allasok'])
    return data
def cb_bnref(*args):
    url = 'https://karrier.bnref.hu/job/filterSubmit'
    r = requests.get(url+'?search%5B_csrf_token%5D=d9e4fd1f7a5025dec40c6dd905fa4234&search%5Bquery%5D=&search%5Blocation%5D=1&search%5Bmainscope%5D=&search%5Bclass%5D=',
                    cookies={'symfony':'6q10os8p2k1lcka9d45fq46931'}) # update before running
    data = []
    if r.status_code == 200:
        rescraper = AutoScraper()
        data = rescraper.build(html=r.text, text_fuzz_ratio=0.7, wanted_list=['/allasajanlat/989/-xiii-kerulet'])
    return data
def cb_smart(*args):
    url = 'https://api.smartrecruiters.com/v1/companies/Eurofins/postings?q=Budapest+IV.&country=hu'
    r = requests.get(url)
    data = []
    if r.status_code == 200:
        data = [row['name'] + ' ~ ' + row['releasedDate'] for row in r.json()['content']]
    return data
def cb_archive(key):
    url = 'https://ghostarchive.org/news/' + key + '/web/ghost/'
    scraper = AutoScraper()
    text = scraper._fetch_html(url, request_args=dict(headers={'Referer':'https://ghostarchive.org/archive/' + key +'?wr=false'}))
    content = unescape(text)
    soup = BeautifulSoup(content if not isinstance(content, str) else unicodedata.normalize("NFKD", content.strip()), "lxml")
    tag = soup.find('input', {'name': 'q'})
    value = tag.attrs['value']
    data = []
    data = scraper.build(html=text, text_fuzz_ratio=0.65, wanted_list=['/Datacenter/Registration/JobAdvertisement///allasok'])
    return data


In [ ]:
links = ['https://karrier.posta.hu/szellemi-poziciok?Y291bnR5JTVCJTVEJTNEQnVkYXBlc3QlMjYuuzzuuzz',
'http://www.allamkincstar.gov.hu/hu/lakossagi-ugyfelek/allashirdetes_/2854',
'https://join.generalicee.com/search/?q=&locationsearch=Budapest&optionsFacetsDD_country=&optionsFacetsDD_city=&optionsFacetsDD_customfield3=&optionsFacetsDD_customfield4=Szak%C3%A9rt%C5%91&optionsFacetsDD_customfield2=Teljes+munkaid%C5%91s&endrow=40&locale=hu_HU&sortColumn=referencedate&sortDirection=desc',
'https://ujvilag.gov.hu/karrier',
'https://nmhh.hu/karrier',
'https://karrier.mnv.hu/DataCenter/Registration/JobAdvertisements/allasok',
'https://karrier.nisz.hu/karrier',
'https://molaris.hu/allasajanlatok',
'https://icell.hrmaster.hu/DataCenter/Registration/JobAdvertisements/allasok',
'https://karrier.bnref.hu',
'https://groupama.karrierportal.hu/allasok?Y29udGVudCUzRGJ1ZGFwZXN0JTI2#!',
'https://www.novodata.hu/rolunk/karrier',
'https://aon.hu/karrier',
'https://jobs.servier.com/egis/search/?createNewAlert=false&q=&locationsearch=Budapest&optionsFacetsDD_customfield4=Digit%C3%A1lis%2C+adat+%C3%A9s+IT',
'https://medicover.hu/blog/allasok/nem-egeszsegugyi-allasok?s=Budapest',
'https://careers.eon.com/magyarorszag/search/?q=-gyakornok+-h%C3%A1l%C3%B3zat+-villanyszerel%C5%91&locationsearch=&optionsFacetsDD_country=&optionsFacetsDD_city=Budapest&optionsFacetsDD_customfield2=',
'https://hrmaster.matusz-vad.hu/DataCenter/Registration/JobAdvertisements/allasok', # 'https://ghostarchive.org/archive/vuwN3'
'https://csalad.hu/rolunk/karrier', # https://csalad.hu/kategoria/karrier
'https://karrier.dkuzrt.hu/DataCenter/Registration/JobAdvertisements/allasok',
'https://careers.eurofins.com/hu/jobs/?countryId=hu&deptid=241,242,243,244,245,246,247,248,250,251',
'https://hellohr.mtu.gov.hu/Datacenter/Registration/JobAdvertisement/allasok', 
'https://hu.eos-solutions.com/career.html',
'https://career.idomsoft.hu/jobs/budapest']

parts = [['https://karrier.posta.hu/allas/projektmenedzser-3160'], ['x'], ['.NET fejlesztő'], ['/karrier/it-software-architect'],
         ['https://nmhh.hu/cikk/234210/IT_tamogatasi_munkatars'],
         ['https://karrier.mnv.hu/Datacenter/Registration/JobAdvertisement/61/kontroller-konyvszakerto/allasok'],
         ['https://karrier.nisz.hu/allasok/incidensmenedzser'], ['#elastic_data_analyst'],
         ['https://icell.hrmaster.hu/Datacenter/Registration/JobAdvertisement/8/qa-engineer/allasok'],
         ['https://karrier.bnref.hu/allasajanlat/875/kontroller-xiii-kerulet'], ['https://groupama.karrierportal.hu/allas/lead-auditor-1655'],
         ['https://www.novodata.hu/nyitott-poziciok/data-engineer-architect'], ['Actuarial Analyst'], ['Data analyst'],
         ['https://medicover.hu/egyeb/kontroller-budapest/','https://medicover.hu/egyeb/aktuarius/','https://medicover.hu/egyeb/elemzo-budapest/'],
         ['Data governance expert'], ['Junior logisztikus'],
         ['https://csalad.hu/rolunk/karrier/szoftverfejleszto'], ['https://karrier.dkuzrt.hu/Datacenter/Registration/JobAdvertisement/79/titkarsagvezeto/allasok'],
         ['x'], ['https://hellohr.mtu.gov.hu/Datacenter/Registration/JobAdvertisement/268/it-desktop-support/allasok'],
         ['Solution Architect / EOS Hungary / Budapest'], ['Közbeszerzési jogi szakértő']] 
wanted_list = parts[0]
callbacks = [TYPE_POST,TYPE_NONE,TYPE_NONE,TYPE_NONE,TYPE_NONE,TYPE_HRMASTER,TYPE_NONE,TYPE_NONE,TYPE_HRMASTER,TYPE_BNREF,TYPE_POST,
             TYPE_NONE,TYPE_NONE,TYPE_NONE,TYPE_NONE,TYPE_NONE,TYPE_HRMASTER,TYPE_NONE,TYPE_HRMASTER,TYPE_SMART,TYPE_HRMASTER, #TYPE_HRMASTER,
             TYPE_NONE,TYPE_NONE]

for u, w, y in it.zip_longest(links, parts, callbacks):
    if w is not None and w != '':
        print(u, w, y)
        append(u, w, y)


In [8]:
try:
    from colorama import Fore, Back, Style
except ImportError:  # fallback so that the imported classes always exist
    class ColorFallback():
        __getattr__ = lambda self, name: ''
    Fore = Back = Style = ColorFallback()

def color_diff(diff):
    for line in diff:
        if line.startswith('+'):
            yield Fore.GREEN + line + Fore.RESET
        elif line.startswith('-'):
            yield Fore.RED + line + Fore.RESET
        elif line.startswith('^'):
            yield Fore.BLUE + line + Fore.RESET
        else:
            yield line


In [10]:
tree = Tree()

@interact(url=links)
def compare(url, update=False, rearchived=''):
    netloc = key(url)
    endpoint = db() + path() + netloc
    if callbacks[links.index(url)] == TYPE_ARCHIVE:
        archive = cb_archive(key(url, 2))[-1]
        endpoint = db() + path() + key(archive)
        netloc = rearchived
    r = requests.get(endpoint)
    asc = AutoScraper()
    asc.stack_list.append(r.json()['autoscraper'])
    if len(asc.stack_list[0]) == 0:
        found = globals()[r.json()['callback']](netloc)
    else:
        pages = gettext(url, pagination='page' if key(url).startswith('medicover') else '')
        found = []
        for page in pages:
            found.append(asc.get_result_similar(html=page))
        found = [item for sublist in found for item in sublist]
    stored = r.json()['result']
    
    for diff in color_diff(dili.unified_diff(stored, found)):
        print(diff)
    if update:
        data = r.json()
        data.update({'result': found})
        data.update({'lastmodified': date.today().isoformat()})
        p = requests.post(endpoint, json=data)
        if p.status_code == 201:
            print(f'modify status {p.status_code}')
    else:
        link = Node()
        link.icon = 'folder'
        link.icon_style = 'warning'
        link.name = key(url)
        link.opened = True
        for line in dili.unified_diff(stored, found):
            if line [1] in ['-', '+', '@']:
                continue
            if link.opened:
                tree.add_node(link)
                link.opened = False
            if line.startswith('-'):
                style = 'danger'
            elif line.startswith('+'):
                style = 'success'
            else:
                style = 'info'
            result = Node()
            result.icon = 'file'
            result.icon_style = style
            result.name = line
            link.add_node(result)

interactive(children=(Dropdown(description='url', options=('https://karrier.posta.hu/szellemi-poziciok?Y291bnR…

In [11]:
lab = JupyterFrontEnd()
panel = Panel(children=[tree])
panel.title.label = 'AutoScraper'
panel.title.icon_class = 'jp-FileIcon'
panel.layout = widgets.Layout(overflow='auto')
tree.layout = widgets.Layout(overflow='auto')
lab.shell.add(panel, 'right', {'rank': 10000})

In [5]:
import ipywidgets as widgets
from IPython.display import display
import time, threading

class Timer():
    def __init__(self, limit=18000):
        self.butt = widgets.Button(description="00:00:00", layout=widgets.Layout(width="120px", height="50px"))
        self.butt.on_click(self.threadTimer)
        self.limit = limit
        self.stopTimer = False
        self.thread = None
        self.buttonState = "Start"
    def timeit(self, limit=180):
        hours = 0
        mins = 0
        secs = 0
        for i in range(1, (limit*60+1)):
            if(self.buttonState == "Start"):
                self.butt.button_style="primary"
            elif(self.buttonState == "Stop"):
                self.butt.button_style="success"
            elif(self.buttonState == "Reset"):
                self.butt.button_style="warning"
            if(self.stopTimer):
                self.stopTimer = False
                self.thread = None
                break
            if i%60 == 0:
                if i%3600 == 0:
                    secs = 0
                    mins = 0
                    hours += 1
                else:
                    secs = 0
                    mins += 1
            else:
                secs += 1
            time.sleep(.0167*100)
            self.butt.description = '{hour:02}:{minute:02}:{second:02}'.format(hour=hours,minute=mins,second=secs)
    def threadTimer(self, b):
        if(self.buttonState == "Start"):
            if(self.thread == None):
                self.thread = threading.Thread(target=self.timeit, args=(self.limit,))
                self.thread.start()
                self.buttonState = "Stop"
        elif(self.buttonState == "Stop"):
            if(self.thread != None):
                self.stopTimer = True
                self.buttonState = "Reset"
        else:
            self.butt.description = '00:00:00'
            self.Thread = None
            self.stopTimer = False
            self.buttonState = "Start"
    def stopTime(self, b):
        self.stopTimer=True
timer = Timer()
display(timer.butt)

Button(description='00:00:00', layout=Layout(height='50px', width='120px'), style=ButtonStyle())